In [1]:
!pip uninstall lightgbm -y
!pip install '/kaggle/input/kaggle-packages/packages/lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl'

Found existing installation: lightgbm 3.3.2
Uninstalling lightgbm-3.3.2:
  Successfully uninstalled lightgbm-3.3.2
Processing /kaggle/input/kaggle-packages/packages/lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl


In [2]:
import lightgbm as lgb
# !pip show lightgbm
from lightgbm import early_stopping, log_evaluation
print(lgb.__version__)

4.1.0


In [3]:
import platform
print(platform.system())

Linux


In [4]:
import pandas as pd
import numpy as np
import gc
from itertools import combinations
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from warnings import simplefilter
import joblib

warnings.filterwarnings('ignore')
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

is_train = False
is_infer = True
N_Folds = 4

In [5]:
# import os
# os.system('mkdir kaggle-optiver-models')

In [6]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

#整体特征
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
max_sizes = train.groupby('stock_id')['bid_size'].max() + train.groupby('stock_id')['ask_size'].max()
min_sizes = train.groupby('stock_id')['bid_size'].min() + train.groupby('stock_id')['ask_size'].min()
mean_sizes = train.groupby('stock_id')['bid_size'].mean() + train.groupby('stock_id')['ask_size'].mean()
first_sizes = train.groupby('stock_id')['bid_size'].first() + train.groupby('stock_id')['ask_size'].first()
last_sizes = train.groupby('stock_id')['bid_size'].last() + train.groupby('stock_id')['ask_size'].last()
#可以再做日期的（好像没看到drop掉日期列）

train = train.dropna(subset=['target'])

In [7]:
# Feature engineering
def feature_eng(df):
    # Remove ['row_id', 'date_id','time_id'].
    cols = [c for c in df.columns if c not in ['row_id', 'date_id','time_id']]
    df = df[cols]
    
    # 匹配失败数量和匹配成功数量的比率
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    # 供需市场的差额
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    # 供需市场总和
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    
    # 供需价格的均值
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    
    # 整体数据情况
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['max_size'] = df['stock_id'].map(max_sizes.to_dict())
    df['min_size'] = df['stock_id'].map(min_sizes.to_dict())
    df['mean_size'] = df['stock_id'].map(mean_sizes.to_dict())
    df['first_size'] = df['stock_id'].map(first_sizes.to_dict())
    df['last_size'] = df['stock_id'].map(last_sizes.to_dict())
    
    # 整体市场规模和当前的市场规模比较
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
    
    prices = ['reference_price', 'far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    # 价格之间做差，做差/求和
    # combinations(range(4), 3) --> (0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]} - {c[1]}) / ({c[0]} + {c[1]})')
        
    for c in combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1) - min_ - max_
        
        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_ - mid_) / (mid_ - min_)
    
    gc.collect()
    
    return df

In [8]:
mae_scores = []
y = train['target'].values
X = feature_eng(train.drop(columns='target'))

y_min = np.min(y)
y_max = np.max(y)

if is_train:
    # 模型在训练集表现的很好 0.45   -> 0.43
    # 模型在验证表现的很好 0.44     -> 0.45

    # 模型是变得更加过拟合了
    # 超参数搜索的工具、gridsearch automl
    params = {
        'learning_rate': 0.018,
        'max_depth': 9,
        'n_estimators': 600,
        'num_leaves': 440,
        'objective': 'mae', # 损失函数
        'random_state': 42, # 随机性
        'reg_alpha': 0.01,
        'reg_lambda': 0.01
        #'device': 'gpu' # 设备
    }
    kf = KFold(n_splits=N_Folds, shuffle=True, random_state=42)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Save more RAM than using pandas dataframe.
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)

        # Early stopping.
        Callbacks = [log_evaluation(period=50), early_stopping(stopping_rounds=50)]
        m = lgb.train(params, train_data, valid_sets=[train_data, valid_data], callbacks=Callbacks)
        print(f"Fold {fold+1} Trainning finished.")

        # Save model.
        model_filename = f"lgb-models-optv2/model_fold_{fold+1}.pkl"
        joblib.dump(m, model_filename)
        y_pred_valid = m.predict(X_valid)

        y_pred_valid = np.nan_to_num(y_pred_valid)
        y_valid = np.nan_to_num(y_valid)
        mae = mean_absolute_error(y_valid, y_pred_valid)
        mae_scores.append(mae)

    # 计算4折平均的MAE
    average_mae = np.mean(mae_scores)
    print(f"4 fold MAE: {average_mae}")

def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out

In [9]:
if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    predictions = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        feat = feature_eng(test)
        fold_prediction = 0
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/kaggle-optiver-models/models-optiver/model_fold_{fold+1}.pkl"
            m = joblib.load(model_filename)
            fold_prediction += m.predict(feat)
        
        fold_prediction /= N_Folds
        fold_prediction = zero_sum(fold_prediction, test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        clipped_predictions = np.clip(fold_prediction, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
